In [1]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
from autogluon.tabular import TabularPredictor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error

In [2]:

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
    

In [3]:
random_grid =  { 
                'randomforest__n_estimators': n_estimators,
               #'max_features': max_features,
               'randomforest__max_depth': max_depth,
               'randomforest__min_samples_split': min_samples_split,
               'randomforest__min_samples_leaf': min_samples_leaf,
               'randomforest__bootstrap': bootstrap
        }

In [4]:
original_data_train = pd.read_csv("train_data.csv",index_col=0)
original_labels_train = pd.read_csv("train_labels.csv",index_col=0)
original_data_test = pd.read_csv("test_data.csv",index_col=0)
original_labels_test = pd.read_csv("test_labels.csv",index_col=0)


In [5]:
original_labels_train = original_labels_train.sort_index()
original_labels_train.reset_index(drop=True, inplace=True)

original_data_train = original_data_train.copy().sort_index()
original_data_train.reset_index(drop=True, inplace=True)

In [6]:
corrupted_data_train = pd.read_csv("train_data_corrupted.csv",index_col=0)
corrupted_data_test = pd.read_csv("test_data_corrupted.csv",index_col=0)


In [103]:
corrupted_data_train = corrupted_data_train.sort_index()
corrupted_data_train.reset_index(drop=True, inplace=True)



In [98]:
original_labels_train.index.equals(corrupted_data_train.index)

True

In [11]:
corrupted_data_train

,V1,V2,V3,V4,V5,V6,V7
2107,3.0,1.0,27.0,23.0,27.0,22.00,35.47
1400,4.0,7.0,25.0,15.0,30.0,9.43,33.54
9117,0.0,0.0,0.0,0.0,20.0,20.00,29.41
5369,5.0,0.0,6.0,18.0,0.0,18.97,NaN
9215,0.0,0.0,0.0,0.0,22.0,22.00,31.62
...,...,...,...,...,...,...,...
5734,0.0,28.0,0.0,0.0,10.0,29.73,NaN
5191,4.0,19.0,21.0,0.0,6.0,24.21,NaN
5390,0.0,0.0,0.0,0.0,36.0,36.00,61.85
860,1.0,22.0,0.0,22.0,7.0,7.00,NaN


In [7]:
def preprocess_and_transform_train(train_data, train_labels, categorical_columns, numerical_columns):
        # Define the categorical preprocessing pipeline
        categorical_preprocessing = Pipeline(
            [
                ('mark_missing', SimpleImputer(strategy='most_frequent')),
                ('one_hot_encode', OneHotEncoder(handle_unknown='ignore',sparse= False))
            ]
        )

        # Define the numerical preprocessing pipeline
        numerical_preprocessing = Pipeline(
            [
                ('mark_missing', SimpleImputer(strategy='mean')),
                ('scaling', StandardScaler())
            ]
        )

        # Define the column transformer
        feature_transformation = ColumnTransformer(transformers=[
                ('categorical_features', categorical_preprocessing, categorical_columns),
                ('scaled_numeric', numerical_preprocessing, numerical_columns)
            ]
        )
        
        # Sort and reset indices for train_labels and train_data
        labels = train_labels.sort_index()
        labels.reset_index(drop=True, inplace=True)

        train_data_sorted = train_data.copy().sort_index()
        train_data_sorted.reset_index(drop=True, inplace=True)

        # Transform the data
        
        if not categorical_columns and not numerical_columns:
            raise ValueError("categorical_columns list and numerical_columns list are empty. Feature Prep. Pipeline needs imput!")
        else:
            transformed_data = pd.DataFrame(feature_transformation.fit_transform(train_data_sorted))
        
        
        
        # Add the labels to the transformed data

        transformed_data["train_label"] = labels
        #transformed_data.to_csv("debug.csv")
        return transformed_data, feature_transformation, labels
    


In [19]:
 transformed_data, feature_transformation, labels = preprocess_and_transform_train(corrupted_data_train.copy(),original_labels_train.copy(),[],["V1","V2","V3","V4","V5","V6","V7"])


In [30]:
transformed_data[6]

0       7.234685e-16
1       7.234685e-16
2       7.234685e-16
3       7.234685e-16
4       7.234685e-16
            ...     
8169    7.234685e-16
8170    7.234685e-16
8171    7.234685e-16
8172    7.234685e-16
8173    7.234685e-16
Name: 6, Length: 8174, dtype: float64

In [10]:
excluded_model_types = ['KNN','CAT','FASTAI','XT','GBM',"RF","NN_TORCH"]

model = TabularPredictor(label="train_label").fit(transformed_data,time_limit=60,excluded_model_types=excluded_model_types,hyperparameters={ 'XGB':{}},feature_generator=None)
print("---------------------------")
print(model.leaderboard())
model.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240904_084731"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels\ag-20240904_084731"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Me

---------------------------
                 model  score_val eval_metric  pred_time_val   fit_time  \
0              XGBoost   0.845966    accuracy       0.139001  13.496633   
1  WeightedEnsemble_L2   0.845966    accuracy       0.140001  13.504630   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.139001          13.496633            1       True   
1                0.001001           0.007997            2       True   

   fit_order  
0          1  
1          2  


In [12]:
def preprocess_and_transform_test(tets_data, test_labels, feature_transformer):
            # Define the categorical preprocessing pipeline
            
            
            # Sort and reset indices for test_labels and tets_data
            labels = test_labels.sort_index()
            labels.reset_index(drop=True, inplace=True)

            tets_data_sorted = tets_data.copy().sort_index()
            tets_data_sorted.reset_index(drop=True, inplace=True)

            # Transform the data

            categorical_columns = []
            numerical_columns =["V1","V2","V3","V4","V5","V6"]
            if not categorical_columns and not numerical_columns:
                 raise ValueError("categorical_columns list and numerical_columns list are empty. Feature Prep. Pipeline needs imput!")
            else:
                transformed_data = pd.DataFrame(feature_transformer.transform(tets_data_sorted))

            
            
            # Add the labels to the transformed data


            return transformed_data, labels



In [21]:
corrupted_test, test_labels_sorted = preprocess_and_transform_test(original_data_test.copy(),original_labels_test.copy(), feature_transformation)

In [25]:
corrupted_test[6].value_counts()

-0.517600    24
 0.393681    21
-0.603129    20
-0.061451    16
 0.001677    16
             ..
-1.380009     1
 2.091006     1
-0.751784     1
 0.149315     1
-0.346544     1
Name: 6, Length: 482, dtype: int64

In [14]:
corrupted_predictions  = model.predict(corrupted_test)

In [15]:
f1_score(test_labels_sorted, corrupted_predictions, average="macro")

0.7567210005150592

In [22]:
corrupted_data_train

,V1,V2,V3,V4,V5,V6,V7
2107,3.0,1.0,27.0,23.0,27.0,22.00,35.47
1400,4.0,7.0,25.0,15.0,30.0,9.43,33.54
9117,0.0,0.0,0.0,0.0,20.0,20.00,29.41
5369,5.0,0.0,6.0,18.0,0.0,18.97,NaN
9215,0.0,0.0,0.0,0.0,22.0,22.00,31.62
...,...,...,...,...,...,...,...
5734,0.0,28.0,0.0,0.0,10.0,29.73,NaN
5191,4.0,19.0,21.0,0.0,6.0,24.21,NaN
5390,0.0,0.0,0.0,0.0,36.0,36.00,61.85
860,1.0,22.0,0.0,22.0,7.0,7.00,NaN


## Imputation experiment

In [129]:
corrupted_data_train = pd.read_csv("train_data_corrupted.csv",index_col=0)

In [130]:
missing_mask = corrupted_data_train["V7"].isna()
df_train= corrupted_data_train[~missing_mask]
df_with_missing = corrupted_data_train[missing_mask]

In [131]:
df_with_missing

,V1,V2,V3,V4,V5,V6,V7
5369,5.0,0.0,6.0,18.0,0.0,18.97,NaN
6041,0.0,18.0,0.0,2.0,9.0,18.36,NaN
4453,0.0,0.0,0.0,0.0,20.0,20.00,NaN
3745,2.0,0.0,18.0,26.0,16.0,26.08,NaN
6946,1.0,0.0,17.0,1.0,33.0,16.03,NaN
...,...,...,...,...,...,...,...
4426,4.0,13.0,33.0,13.0,27.0,6.00,NaN
466,4.0,17.0,12.0,17.0,18.0,6.00,NaN
5734,0.0,28.0,0.0,0.0,10.0,29.73,NaN
5191,4.0,19.0,21.0,0.0,6.0,24.21,NaN


In [132]:
y= 
X= 

In [144]:
df_train.iloc[:,:-1]

,V1,V2,V3,V4,V5,V6
2107,3.0,1.0,27.0,23.0,27.0,22.00
1400,4.0,7.0,25.0,15.0,30.0,9.43
9117,0.0,0.0,0.0,0.0,20.0,20.00
9215,0.0,0.0,0.0,0.0,22.0,22.00
6492,1.0,0.0,23.0,6.0,20.0,6.71
...,...,...,...,...,...,...
1685,1.0,0.0,24.0,10.0,18.0,11.66
8322,3.0,0.0,19.0,1.0,31.0,12.04
6265,4.0,12.0,8.0,6.0,4.0,7.21
5390,0.0,0.0,0.0,0.0,36.0,36.00


In [149]:
 df_train_imp, feature_transformation, labels = preprocess_and_transform_train(df_train.iloc[:,:-1],df_train.iloc[:,-1:],[],["V1","V2","V3","V4","V5","V6"])


In [151]:
excluded_model_types = ['KNN','CAT','FASTAI','XT','GBM',"RF","NN_TORCH"]


imputer = TabularPredictor(label="train_label").fit(df_train_imp,time_limit=360,presets="high_quality",excluded_model_types=excluded_model_types,hyperparameters={ 'XGB':{}},feature_generator=None)
print("---------------------------")
print(imputer.leaderboard())
imputer.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240830_154130"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 360 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240830_154130/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 25 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 335 seconds.
Starting full fit now with num_stack_level

---------------------------
                      model  score_val              eval_metric  \
0            XGBoost_BAG_L1  -4.167295  root_mean_squared_error   
1       WeightedEnsemble_L2  -4.167295  root_mean_squared_error   
2       XGBoost_BAG_L1_FULL        NaN  root_mean_squared_error   
3  WeightedEnsemble_L2_FULL        NaN  root_mean_squared_error   

   pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  \
0       0.502948  5.063996                0.502948           5.063996   
1       0.503948  5.066996                0.001000           0.003000   
2            NaN  1.388998                     NaN           1.388998   
3            NaN  1.391999                     NaN           0.003000   

   stack_level  can_infer  fit_order  
0            1      False          1  
1            2      False          2  
2            1       True          3  
3            2       True          4  


In [25]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
pip = Pipeline(steps=[

    ("scale", StandardScaler()),
    ("randomforest", rf)
])
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = pip, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\Petro\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('scale', StandardScaler()),
                                             ('randomforest',
                                              RandomForestRegressor())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'randomforest__bootstrap': [True,
                                                                    False],
                                        'randomforest__max_depth': [10, 20, 30,
                                                                    40, 50, 60,
                                                                    70, 80, 90,
                                                                    100, 110,
                                                                    None],
                                        'randomforest__min_samples_leaf': [1, 2,
                                                                           4],
                                        'randomforest__min_samples_split': [2,
                                                                            5,
                                                                            10],
                                        'randomforest__n_estimators': [10, 152,
                                                                       294, 436,
                                                                       578, 720,
                                                                       862,
                                                                       1005,
                                                                       1147,
                                                                       1289,
                                                                       1431,
                                                                       1573,
                                                                       1715,
                                                                       1857,
                                                                       2000]},
                   random_state=42, verbose=2)

In [26]:
randomForest= rf_random.best_estimator_

In [122]:
corrupted_data_train.loc[missing_mask].iloc[:,:-1]

0       32.231998
1       27.535999
2       34.824001
3       29.518000
4       20.548000
          ...    
8169    56.127998
8170    58.172001
8171    58.632000
8172    39.292000
8173    58.790001
Name: V7, Length: 4087, dtype: float64

In [152]:
missing_mask = corrupted_data_train["V7"].isna()
amount_missing_in_columns = missing_mask.sum()
column= "V7"
corrupted_data_train.loc[missing_mask, column] = imputer.predict(corrupted_data_train.loc[missing_mask].iloc[:,:-1])

In [135]:
#prediction = imputer.predict(df_with_missing.iloc[:,:-1])

In [136]:
#df_with_missing["V7"] = prediction
#df_new_imputed = df_train._append(df_with_missing)

In [137]:
corrupted_data_train

,V1,V2,V3,V4,V5,V6,V7
2107,3.0,1.0,27.0,23.0,27.0,22.00,35.470000
1400,4.0,7.0,25.0,15.0,30.0,9.43,33.540000
9117,0.0,0.0,0.0,0.0,20.0,20.00,29.410000
5369,5.0,0.0,6.0,18.0,0.0,18.97,46.152866
9215,0.0,0.0,0.0,0.0,22.0,22.00,31.620000
...,...,...,...,...,...,...,...
5734,0.0,28.0,0.0,0.0,10.0,29.73,63.167416
5191,4.0,19.0,21.0,0.0,6.0,24.21,47.628090
5390,0.0,0.0,0.0,0.0,36.0,36.00,61.850000
860,1.0,22.0,0.0,22.0,7.0,7.00,36.602909


In [153]:
transformed_data, feature_transformation, labels = preprocess_and_transform_train(corrupted_data_train.copy(),original_labels_train.copy(),[],["V1","V2","V3","V4","V5","V6","V7"])


In [154]:
excluded_model_types = ['KNN','CAT','FASTAI','XT','GBM',"RF","NN_TORCH"]


models = TabularPredictor(label="train_label").fit(transformed_data,time_limit=60,excluded_model_types=excluded_model_types,hyperparameters={ 'XGB':{}},feature_generator=None)
print("---------------------------")
print(models.leaderboard())
models.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240830_154213"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels\ag-20240830_154213"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Me

---------------------------
                 model  score_val eval_metric  pred_time_val  fit_time  \
0              XGBoost   0.830073    accuracy       0.138000  9.242995   
1  WeightedEnsemble_L2   0.830073    accuracy       0.138999  9.247997   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.138000           9.242995            1       True   
1                0.000999           0.005002            2       True   

   fit_order  
0          1  
1          2  


In [155]:
imputed_predictions  = models.predict(corrupted_test)

In [156]:
f1_score(test_labels_sorted, imputed_predictions, average="macro")

0.6960925545931959

In [86]:
excluded_model_types = ['KNN', 'CAT','FASTAI','XT','GBM','RF','NN_TORCH']


model = TabularPredictor(label="V7").fit(df_new,time_limit=360,excluded_model_types=excluded_model_types,hyperparameters={'XGB':{}},feature_generator=None,ag_args_fit={'num_gpus': 1},ag_args_ensemble =dict(fold_fitting_strategy='sequential_local'))
print("---------------------------")
print(model.leaderboard())


model.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels/ag-20240828_153907"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #44~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Jun 18 14:36:16 UTC 2
CPU Count:          16
Memory Avail:       52.54 GB / 62.01 GB (84.7%)
Disk Space Avail:   1648.32 GB / 1832.21 GB (90.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='med

---------------------------
                 model  score_val              eval_metric  pred_time_val  \
0              XGBoost  -3.504626  root_mean_squared_error       0.004656   
1  WeightedEnsemble_L2  -3.504626  root_mean_squared_error       0.004881   

   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  0.943475                0.004656           0.943475            1   
1  0.944741                0.000226           0.001266            2   

   can_infer  fit_order  
0       True          1  
1       True          2  
